In [1]:
# Import libs
import os
import datetime
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# Load dataset
# get data_dir for tranining
current_path = os.getcwd()

img_height = 224
img_width = 224
batch_size = 64

# set train dataset
train_data_dir = os.path.join(current_path, 'Vegetable_Images\\train')
train_ds = keras.utils.image_dataset_from_directory(
    train_data_dir,
    label_mode='categorical',
    image_size = (img_height, img_width),
    batch_size = batch_size
)

# set validation dataset
val_data_dir = os.path.join(current_path, 'Vegetable_Images\\validation')
val_ds = keras.utils.image_dataset_from_directory(
    val_data_dir,
    label_mode='categorical',
    image_size = (img_height, img_width),
    batch_size = batch_size
)

class_names = train_ds.class_names
class_num = len(class_names)

Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.


In [3]:
# Normalize data
data_scaling = keras.layers.Rescaling(scale=1./255)

train_ds = train_ds.map( lambda x, y: (data_scaling(x), y))
val_ds = val_ds.map( lambda x, y: (data_scaling(x), y))

In [4]:
# Build model
BfS_Model = keras.Sequential([
    keras.layers.Input(shape=(img_height, img_width, 3)),
    keras.layers.Resizing(height=32, width=32),
    
    keras.layers.Conv2D(filters=32, kernel_size=3, strides=(1,1), padding='same', activation='relu', use_bias=True),
    keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='valid'),
    
    keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding='same', activation='relu', use_bias=True),
    keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='valid'),

    keras.layers.Conv2D(filters=128, kernel_size=3, strides=(1,1), padding='same', activation='relu', use_bias=True),
    keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='valid'),    

    keras.layers.Conv2D(filters=256, kernel_size=3, strides=(1,1), padding='same', activation='relu', use_bias=True),
    keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='valid'),

    keras.layers.Flatten(),

    keras.layers.Dense(units=1024, activation='relu', use_bias=True),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(units=class_num, activation='softmax', use_bias=True)
])

BfS_Model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

BfS_Model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 4, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │        15,375 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,453,391 (5.54 MB)

 Trainable params: 1,453,391 (5.54 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Train model
logdir = os.path.join("logs", datetime.datetime.now().strftime("BfS_Model-%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

history = BfS_Model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[tensorboard_callback]
)

Epoch 1/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 13s 44ms/step - accuracy: 0.1937 - loss: 2.4244 - val_accuracy: 0.4677 - val_loss: 1.5743
Epoch 2/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.5109 - loss: 1.4761 - val_accuracy: 0.6420 - val_loss: 1.1365
Epoch 3/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - accuracy: 0.6287 - loss: 1.1263 - val_accuracy: 0.7083 - val_loss: 0.9061
Epoch 4/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.7130 - loss: 0.8835 - val_accuracy: 0.7770 - val_loss: 0.7095
Epoch 5/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.7751 - loss: 0.7072 - val_accuracy: 0.7947 - val_loss: 0.6406
Epoch 6/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.8048 - loss: 0.6136 - val_accuracy: 0.8250 - val_loss: 0.5684
Epoch 7/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - accuracy: 0.8286 - loss: 0.5311 - val_accuracy: 0.8500 - val_loss: 0.4861
Epoch 8/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - accuracy: 0.8542 - loss: 0.4592 - 

In [6]:
# Save model as keras
BfS_Model.save('saved_models\BfS_Model.keras')